##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import

In [ ]:
!pip install -U -q google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 11.7 MB/s eta 0:00:00


In [ ]:
# Install the client library and import necessary modules.
import google.generativeai as genai

import base64
import copy
import hashlib
import io
import json
import mimetypes
import pathlib
import pprint
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

### Mount Google Drive

In [ ]:
from google.colab import drive

drive.mount("/gdrive")

Mounted at /gdrive


## Set the API key

Add your API_KEY to the secrets manager in the left panel "🔑".

In [ ]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

In [ ]:
# Configure the client library by providing your API key.
genai.configure(api_key=GOOGLE_API_KEY)

## Parse the arguments

In [ ]:
model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = 'W3sicGFydHMiOlt7InRleHQiOiJpbnB1dDogSGksIEknbSByZWFsbHkgYW54aW91cyBhYm91dCBteSB1cGNvbWluZyBleGFtcy4gSSBmZWVsIGxpa2UgSSBjYW4ndCBjb25jZW50cmF0ZSBhbmQgaXQncyBvdmVyd2hlbG1pbmcuIn0seyJ0ZXh0Ijoib3V0cHV0OiBJJ20gcmVhbGx5IHNvcnJ5IHlvdSdyZSBmZWVsaW5nIHRoaXMgd2F5IHJpZ2h0IG5vdy4gRXhhbSBzdHJlc3MgY2FuIGJlIHRvdWdoLCBidXQgaXQncyBpbXBvcnRhbnQgdG8gdGFrZSB0aGluZ3Mgb25lIHN0ZXAgYXQgYSB0aW1lLiBIYXZlIHlvdSB0cmllZCBhbnkgcmVsYXhhdGlvbiB0ZWNobmlxdWVzLCBsaWtlIGRlZXAgYnJlYXRoaW5nIG9yIGEgc2hvcnQgYnJlYWsgdG8gY2xlYXIgeW91ciBtaW5kPyBJdCdzIGFsc28gaGVscGZ1bCB0byBjcmVhdGUgYSBzbWFsbCBzdHVkeSBzY2hlZHVsZSBzbyB0aGF0IHlvdSBjYW4gZm9jdXMgb24gb25lIHRhc2sgYXQgYSB0aW1lLiBIb3cgYXJlIHlvdSBmZWVsaW5nIGFib3V0IHlvdXIgc3R1ZHkgcHJvZ3Jlc3Mgc28gZmFyPyJ9LHsidGV4dCI6ImlucHV0OiBJJ3ZlIGJlZW4gZmVlbGluZyByZWFsbHkgbG9uZWx5IGF0IGNvbGxlZ2UuIEkgZG9u4oCZdCBrbm93IGhvdyB0byBtYWtlIG5ldyBmcmllbmRzLCBhbmQgaXTigJlzIHN0YXJ0aW5nIHRvIGFmZmVjdCBtZS4ifSx7InRleHQiOiJvdXRwdXQ6IEknbSByZWFsbHkgc29ycnkgeW91J3JlIGZlZWxpbmcgbG9uZWx5LiBDb2xsZWdlIGNhbiBmZWVsIGlzb2xhdGluZywgZXNwZWNpYWxseSBpZiBpdCdzIGhhcmQgdG8gY29ubmVjdCB3aXRoIG90aGVycyBhdCBmaXJzdC4gU29tZXRpbWVzIGpvaW5pbmcgYSBjbHViIG9yIHN0dWR5IGdyb3VwIGNhbiBiZSBhIGdvb2Qgd2F5IHRvIG1lZXQgcGVvcGxlIHdobyBzaGFyZSBzaW1pbGFyIGludGVyZXN0cy4gSXQgY2FuIGFsc28gaGVscCB0byByZWFjaCBvdXQgdG8gY2xhc3NtYXRlc+KAlHNvbWV0aW1lcywgb3RoZXJzIGZlZWwgdGhlIHNhbWUgd2F5IGJ1dCBhcmVuJ3Qgc3VyZSBob3cgdG8gc3RhcnQgYSBjb252ZXJzYXRpb24uIFdvdWxkIHlvdSBsaWtlIHNvbWUgaWRlYXMgZm9yIGNsdWJzIG9yIGdyb3VwcyB0aGF0IHlvdSBjb3VsZCBleHBsb3JlPyJ9LHsidGV4dCI6ImlucHV0OiBJJ20gZHJvd25pbmcgaW4gYXNzaWdubWVudHMuIEkgZG9uJ3Qga25vdyBob3cgdG8ga2VlcCB1cCwgYW5kIEkgZmVlbCBsaWtlIEknbSBmYWxsaW5nIGJlaGluZC4ifSx7InRleHQiOiJvdXRwdXQ6IEl0IHNvdW5kcyBsaWtlIHlvdSdyZSBjYXJyeWluZyBhIGxvdCByaWdodCBub3csIGFuZCBpdCdzIHVuZGVyc3RhbmRhYmxlIHRvIGZlZWwgb3ZlcndoZWxtZWQuIEJyZWFraW5nIHRhc2tzIGRvd24gaW50byBzbWFsbGVyIHBpZWNlcyBtaWdodCBoZWxw4oCUY2FuIHlvdSBmb2N1cyBvbiBvbmUgYXNzaWdubWVudCBhdCBhIHRpbWU/IEFsc28sIGRvbuKAmXQgaGVzaXRhdGUgdG8gYXNrIHlvdXIgcHJvZmVzc29ycyBvciBwZWVycyBmb3IgY2xhcmlmaWNhdGlvbiBvciBleHRlbnNpb25zIGlmIG5lZWRlZC4gSXTigJlzIG9rYXkgdG8gc2VlayBoZWxwLiBEbyB5b3Ugd2FudCBzb21lIHRpcHMgb24gbWFuYWdpbmcgeW91ciB0aW1lLCBvciBtYXliZSBhIGd1aWRlIHRvIHByaW9yaXRpemUgeW91ciB0YXNrcz8ifSx7InRleHQiOiJpbnB1dDogSSBqdXN0IGZlZWwgc28gZG93biBsYXRlbHksIGFuZCBJIGRvbid0IGhhdmUgdGhlIGVuZXJneSB0byBkbyBhbnl0aGluZyBhbnltb3JlLiBJIGRvbuKAmXQga25vdyB3aGF04oCZcyB3cm9uZyB3aXRoIG1lLiJ9LHsidGV4dCI6Im91dHB1dDogSSdtIHJlYWxseSBzb3JyeSB5b3UncmUgZmVlbGluZyB0aGlzIHdheS4gSXQncyBoYXJkIHdoZW4geW91J3JlIGdvaW5nIHRocm91Z2ggcGVyaW9kcyBvZiBsb3cgZW5lcmd5IGFuZCBzYWRuZXNzLiBJdCBtaWdodCBiZSBoZWxwZnVsIHRvIHJlYWNoIG91dCB0byBzb21lb25lIHlvdSB0cnVzdCwgbGlrZSBhIGZyaWVuZCBvciBjb3Vuc2Vsb3IsIHRvIHRhbGsgYWJvdXQgd2hhdCB5b3UncmUgZXhwZXJpZW5jaW5nLiBTb21ldGltZXMgdGFraW5nIHNtYWxsIHN0ZXBzLCBsaWtlIGRvaW5nIHNvbWV0aGluZyB5b3UgZW5qb3kgb3IganVzdCBnZXR0aW5nIG91dHNpZGUgZm9yIGEgc2hvcnQgd2FsaywgY2FuIG1ha2UgYSBkaWZmZXJlbmNlLiBJ4oCZbSBoZXJlIGZvciB5b3XigJRkbyB5b3Ugd2FudCB0byB0YWxrIG1vcmUgYWJvdXQgd2hhdOKAmXMgYmVlbiBvbiB5b3VyIG1pbmQ/In0seyJ0ZXh0IjoiaW5wdXQ6IEkgbWlzcyBob21lIHNvIG11Y2guIEkgdGhvdWdodCBJIHdvdWxkIGFkanVzdCB0byBiZWluZyBhd2F5LCBidXQgSSBmZWVsIG1vcmUgaG9tZXNpY2sgZXZlcnkgZGF5LiJ9LHsidGV4dCI6Im91dHB1dDogSSdtIHJlYWxseSBzb3JyeSB5b3UncmUgZmVlbGluZyBob21lc2ljay4gSXTigJlzIG5hdHVyYWwgdG8gbWlzcyBob21lLCBlc3BlY2lhbGx5IHdoZW4geW91J3JlIGFkanVzdGluZyB0byBhIG5ldyBlbnZpcm9ubWVudC4gU29tZXRpbWVzIHN0YXlpbmcgY29ubmVjdGVkIHdpdGggZmFtaWx5IGFuZCBmcmllbmRzIGZyb20gaG9tZSwgbGlrZSBzY2hlZHVsaW5nIHJlZ3VsYXIgY2FsbHMsIGNhbiBoZWxwIHlvdSBmZWVsIG1vcmUgZ3JvdW5kZWQuIEhhdmUgeW91IGhhZCB0aGUgY2hhbmNlIHRvIGV4cGxvcmUgdGhpbmdzIGFyb3VuZCB5b3VyIG5ldyBhcmVhIHRoYXQgY291bGQgbWFrZSBpdCBmZWVsIG1vcmUgbGlrZSBob21lPyJ9LHsidGV4dCI6ImlucHV0OiBJIGZlZWwgZXhoYXVzdGVkIGFsbCB0aGUgdGltZSwgbGlrZSBubyBtYXR0ZXIgaG93IG11Y2ggSSByZXN0LCBJIGNhbid0IGdldCBteSBlbmVyZ3kgYmFjay4gSSB0aGluayBJJ20gYnVybnQgb3V0LiJ9LHsidGV4dCI6Im91dHB1dDogSSdtIHJlYWxseSBzb3JyeSB5b3UncmUgZmVlbGluZyB0aGlzIHdheS4gQnVybm91dCBjYW4gaGFwcGVuIHdoZW4geW914oCZdmUgYmVlbiBqdWdnbGluZyBhIGxvdCBmb3IgYSBsb25nIHRpbWUuIEl04oCZcyBpbXBvcnRhbnQgdG8gZ2l2ZSB5b3Vyc2VsZiBwZXJtaXNzaW9uIHRvIHJlc3QgYW5kIHRha2UgY2FyZSBvZiB5b3VyIG1lbnRhbCBhbmQgcGh5c2ljYWwgaGVhbHRoLiBFdmVuIHNob3J0IGJyZWFrcyBvciBkb2luZyBzb21ldGhpbmcgZW5qb3lhYmxlIG1pZ2h0IGhlbHAgcmVjaGFyZ2UgeW91ciBlbmVyZ3kuIEhhdmUgeW91IGNvbnNpZGVyZWQgdGFsa2luZyB0byBzb21lb25lIGFib3V0IHlvdXIgd29ya2xvYWQgb3IgZmluZGluZyB3YXlzIHRvIHNldCBib3VuZGFyaWVzIGZvciB5b3Vyc2VsZj8ifSx7InRleHQiOiJpbnB1dDogSSBjYW7igJl0IGZvY3VzIG9uIGFueXRoaW5nIGFueW1vcmUuIEkgdHJ5IHRvIHN0dWR5LCBidXQgbXkgbWluZCBqdXN0IGtlZXBzIHdhbmRlcmluZy4ifSx7InRleHQiOiJvdXRwdXQ6IEl0IHNvdW5kcyBmcnVzdHJhdGluZyB0byBmZWVsIHRoYXQgd2F5LiBTb21ldGltZXMgd2hlbiBvdXIgbWluZHMgYXJlIG92ZXJsb2FkZWQsIGl0J3MgaGFyZGVyIHRvIGNvbmNlbnRyYXRlLiBJdCBtaWdodCBoZWxwIHRvIHRha2Ugc21hbGwgYnJlYWtzIG9yIHRyeSB0ZWNobmlxdWVzIGxpa2UgdGhlIFBvbW9kb3JvIG1ldGhvZOKAlHdvcmtpbmcgaW4gc2hvcnQgaW50ZXJ2YWxzIHdpdGggYnJlYWtzIGluIGJldHdlZW4uIEhhdmUgeW91IGJlZW4gZ2V0dGluZyBlbm91Z2ggc2xlZXAgYW5kIHRha2luZyBjYXJlIG9mIHlvdXJzZWxmPyBTZWxmLWNhcmUgY2FuIHJlYWxseSBpbXBhY3QgZm9jdXMuIFdoYXQgdXN1YWxseSBoZWxwcyB5b3UgcmVmb2N1cyB3aGVuIHlvdSdyZSBkaXN0cmFjdGVkPyJ9LHsidGV4dCI6ImlucHV0OiBJ4oCZbSBzY2FyZWQgb2YgZmFpbGluZy4gTm8gbWF0dGVyIGhvdyBtdWNoIEkgc3R1ZHksIEkgc3RpbGwgZmVlbCBsaWtlIEnigJltIGdvaW5nIHRvIG1lc3MgdXAuIn0seyJ0ZXh0Ijoib3V0cHV0OiBJdCdzIHVuZGVyc3RhbmRhYmxlIHRvIGZlZWwgdGhhdCB3YXksIGVzcGVjaWFsbHkgd2hlbiB5b3UgY2FyZSBhYm91dCB5b3VyIHJlc3VsdHMuIEJ1dCByZW1lbWJlciwgZmFpbHVyZSBpcyBhIHBhcnQgb2YgbGVhcm5pbmcsIGFuZCBpdCBkb2Vzbid0IGRlZmluZSB5b3VyIHdvcnRoIG9yIGFiaWxpdGllcy4gWW91J3JlIHRyeWluZyB5b3VyIGJlc3QsIGFuZCB0aGF0IGVmZm9ydCBjb3VudHMgZm9yIGEgbG90LiBXb3VsZCB5b3UgbGlrZSB0byB0YWxrIGFib3V0IHdoYXQncyBtYWtpbmcgeW91IGZlZWwgbW9zdCB1bmNlcnRhaW4sIG9yIHNvbWUgd2F5cyB0byBidWlsZCBjb25maWRlbmNlIGluIHlvdXIgcHJlcGFyYXRpb24/In0seyJ0ZXh0IjoiaW5wdXQ6IEkga2VlcCBwdXR0aW5nIG9mZiBteSB3b3JrLCBhbmQgbm93IEnigJltIHdvcnJpZWQgSSB3b27igJl0IGJlIGFibGUgdG8gZmluaXNoIGl0IGFsbC4ifSx7InRleHQiOiJvdXRwdXQ6IFByb2NyYXN0aW5hdGlvbiBjYW4gc25lYWsgdXAgb24gYWxsIG9mIHVzLCBlc3BlY2lhbGx5IHdoZW4gdGFza3MgZmVlbCBvdmVyd2hlbG1pbmcuIFNvbWV0aW1lcyBzdGFydGluZyB3aXRoIHRoZSBzbWFsbGVzdCwgZWFzaWVzdCB0YXNrIGNhbiBidWlsZCBtb21lbnR1bS4gWW91IGNhbiBhbHNvIHRyeSBzZXR0aW5nIGEgdGltZXIgZm9yIGp1c3QgMTAtMTUgbWludXRlcyB0byB3b3JrIG9uIHNvbWV0aGluZ+KAlGl0IG1pZ2h0IGhlbHAgeW91IGdldCBpbnRvIHRoZSBmbG93LiBIb3cgYWJvdXQgd2UgbWFrZSBhIHBsYW4gdG9nZXRoZXIgdG8gZ2V0IHN0YXJ0ZWQ/In0seyJ0ZXh0IjoiaW5wdXQ6IEkgZG9u4oCZdCBmZWVsIGxpa2UgSeKAmW0gZ29vZCBlbm91Z2gsIG5vIG1hdHRlciB3aGF0IEkgZG8uIEV2ZXJ5b25lIGVsc2Ugc2VlbXMgc28gbXVjaCBtb3JlIGNhcGFibGUgdGhhbiBtZS4ifSx7InRleHQiOiJvdXRwdXQ6IEknbSByZWFsbHkgc29ycnkgeW91J3JlIGZlZWxpbmcgbGlrZSB0aGlzLiBJdCdzIHRvdWdoIHdoZW4gd2UgY29tcGFyZSBvdXJzZWx2ZXMgdG8gb3RoZXJzLCBidXQgcmVtZW1iZXIsIGV2ZXJ5b25lIGhhcyB0aGVpciBvd24gc3RydWdnbGVzIHRoYXQgYXJlbid0IGFsd2F5cyB2aXNpYmxlLiBZb3XigJlyZSBjYXBhYmxlIGluIHlvdXIgb3duIHdheSwgYW5kIHlvdXIgam91cm5leSBpcyB1bmlxdWUuIFdvdWxkIHlvdSBsaWtlIHRvIHRhbGsgYWJvdXQgd2hhdOKAmXMgYmVlbiBtYWtpbmcgeW91IGZlZWwgdGhpcyB3YXk/IEnigJltIGhlcmUgdG8gbGlzdGVuLiJ9LHsidGV4dCI6ImlucHV0OiBpbSBzdHJlc3NlZCBhYm91dCBub3QgZ2V0dGluZyBnb29kIGdyYWRlcyBpbiBteSBtaWRzZW0gZXhhbXMgdGhhdCBpIHdyb3RlLiBjYW4gd2UgYnJhaW5zdG9ybSBzb21lIHdheXMgdG8gbWFuYWdlIGl0In0seyJ0ZXh0Ijoib3V0cHV0OiAifV19XQ==' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo2NCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

In [ ]:
gais_contents

[{'parts': [{'text': "input: Hi, I'm really anxious about my upcoming exams. I feel like I can't concentrate and it's overwhelming."},
   {'text': "output: I'm really sorry you're feeling this way right now. Exam stress can be tough, but it's important to take things one step at a time. Have you tried any relaxation techniques, like deep breathing or a short break to clear your mind? It's also helpful to create a small study schedule so that you can focus on one task at a time. How are you feeling about your study progress so far?"},
   {'text': "input: I've been feeling really lonely at college. I don’t know how to make new friends, and it’s starting to affect me."},
   {'text': "output: I'm really sorry you're feeling lonely. College can feel isolating, especially if it's hard to connect with others at first. Sometimes joining a club or study group can be a good way to meet people who share similar interests. It can also help to reach out to classmates—sometimes, others feel the same

In [ ]:
generation_config

{'temperature': 1, 'top_p': 0.95, 'top_k': 64, 'max_output_tokens': 8192}

In [ ]:
safety_settings

{}

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [ ]:
# @title `show_file` function
def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")

In [ ]:
for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

input: Hi, I'm really anxious about my upcoming exams. I feel like I can't concentrate and it's overwhelming. 

output: I'm really sorry you're feeling this way right now. Exam stress can be tough, but it's important to take things one step at a time. Have you tried any relaxation techniques, like deep breathing or a short break to clear your mind? It's also helpful to create a small study schedule so that you can focus on one task at a time. How are you feeling about your study progress so far? 

input: I've been feeling really lonely at college. I don’t know how to make new friends, and it’s starting to affect me. 

output: I'm really sorry you're feeling lonely. College can feel isolating, especially if it's hard to connect with others at first. Sometimes joining a club or study group can be a good way to meet people who share similar interests. It can also help to reach out to classmates—sometimes, others feel the same way but aren't sure how to start a conversation. Would you like

## Convert & upload files

For each file, Google AI Studio either sent:
- a Google Drive IDs
- a URL, or
- the raw bytes (`inline_data`).

The API itself onlty understands two ways of sending files:

- `inline_data` - where the bytes are placed inline in the request.
- `file_data` - where the file is uploaded to the Files API, and you pass a reference to that file.

This section goes through the `contents` from Google AI Studio, and uploads the file data, to the Files API, so Gemini can access it:

In [ ]:
# @title `upload_file` function

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


def upload_file_data(file_data):
    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        hash = hashlib.sha256(data).hexdigest()
        path = tempfiles / hash
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files"in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

In [ ]:
contents = copy.deepcopy(gais_contents)

for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data)

Here is the coverted `Content`s:

In [ ]:
contents

[{'parts': [{'text': "input: Hi, I'm really anxious about my upcoming exams. I feel like I can't concentrate and it's overwhelming."},
   {'text': "output: I'm really sorry you're feeling this way right now. Exam stress can be tough, but it's important to take things one step at a time. Have you tried any relaxation techniques, like deep breathing or a short break to clear your mind? It's also helpful to create a small study schedule so that you can focus on one task at a time. How are you feeling about your study progress so far?"},
   {'text': "input: I've been feeling really lonely at college. I don’t know how to make new friends, and it’s starting to affect me."},
   {'text': "output: I'm really sorry you're feeling lonely. College can feel isolating, especially if it's hard to connect with others at first. Sometimes joining a club or study group can be a good way to meet people who share similar interests. It can also help to reach out to classmates—sometimes, others feel the same

## Call `generate_content`

In [ ]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=False,
)

In [ ]:
if generation_config.get("candidate_count", 1) == 1:
    display(Markdown(response.text))

It’s understandable to feel stressed about exams, especially when you’ve just taken them. Let’s brainstorm some ways to manage this stress:

**1. Acknowledge and Accept:** First, it’s important to acknowledge your feelings. It’s okay to feel stressed, anxious, or even disappointed. Try to accept that you did your best, and the results are out of your control now.

**2. Focus on What You Can Control:** You can't change the past, but you can focus on the present.  Instead of dwelling on what might have been, shift your attention to things you can control:

* **Review the material:** If you have any feedback from the exams, review it carefully and identify areas where you can improve.
* **Plan for future exams:**  If there are more exams coming up, make a plan to study more effectively and proactively address areas you struggled with.
* **Take care of yourself:**  Prioritize self-care activities like exercise, healthy eating, getting enough sleep, and spending time doing things you enjoy.

**3. Seek Support:**  Don't hesitate to talk to someone you trust about how you're feeling. This could be a friend, family member, or a counselor. They can provide a listening ear and offer support.

**4. Practice Relaxation Techniques:** Stress management techniques like deep breathing, meditation, or progressive muscle relaxation can be helpful in reducing anxiety. 

**5. Shift Your Perspective:**  Remember that exams are just one part of your overall journey.  Try to focus on your strengths, accomplishments, and the bigger picture.  

**Do any of these ideas resonate with you?  What do you think might be helpful to try right now?**


In [ ]:
response.candidates

[content {
  parts {
    text: "It\342\200\231s understandable to feel stressed about exams, especially when you\342\200\231ve just taken them. Let\342\200\231s brainstorm some ways to manage this stress:\n\n**1. Acknowledge and Accept:** First, it\342\200\231s important to acknowledge your feelings. It\342\200\231s okay to feel stressed, anxious, or even disappointed. Try to accept that you did your best, and the results are out of your control now.\n\n**2. Focus on What You Can Control:** You can\'t change the past, but you can focus on the present.  Instead of dwelling on what might have been, shift your attention to things you can control:\n\n* **Review the material:** If you have any feedback from the exams, review it carefully and identify areas where you can improve.\n* **Plan for future exams:**  If there are more exams coming up, make a plan to study more effectively and proactively address areas you struggled with.\n* **Take care of yourself:**  Prioritize self-care activitie

## Or Create a chat

In [ ]:
gemini = genai.GenerativeModel(
    model_name=model,
    generation_config=generation_config,
    safety_settings=safety_settings,
)

A `ChatSession`, should always end with the model's turn.

So before creating the `chat` check whos turn was last.

If the user was last, attach all but the last content as the `history` and send the last content with `send_message` to get the model's response.

If the model was last, put the whole contents list in the history.

In [ ]:
model_turn = contents[-1].get("role", None) == "user"

if model_turn:
    chat = gemini.start_chat(history=contents[:-1])

    response = chat.send_message(contents[-1])

    if generation_config.get("candidate_count", 1) == 1:
        display(Markdown(response.text))
else:
    chat = gemini.start_chat(history=contents)

In [ ]:
if model_turn:
    response.candidates

After that use `send_message` to continue the conversation.

In [ ]:
# Ensure the last turn in the history has a valid role before starting the chat
if not contents[-1].get("role", None) in ["user", "model"]:
    contents[-1]["role"] = "user"  # Assign "user" if no role is found

if model_turn:
    chat = gemini.start_chat(history=contents[:-1])
    response = chat.send_message(contents[-1])

    if generation_config.get("candidate_count", 1) == 1:
        display(Markdown(response.text))
else:
    chat = gemini.start_chat(history=contents)

In [ ]:
 response = chat.send_message("Interesting, tell me more.")
 display(Markdown(response.text))

Okay, let's delve deeper! I'm ready to explore more specific ways to address your stress about the mid-semester exams. To give you the best advice, I need a little more context. 

Tell me, what specifically are you feeling stressed about?  For example: 

* **Is it about the content itself?** Do you feel unprepared for certain topics or concepts? 
* **Is it about the exam format?** Are you worried about multiple-choice questions, essays, or other types of assessment?
* **Is it about time management?**  Do you feel overwhelmed by the amount of material you need to cover?
* **Is it about your performance?** Are you concerned about not doing well enough or not meeting your own expectations?

Once I understand your specific concerns, I can offer more tailored advice.  I can help you:

* **Break down the material into manageable chunks.**
* **Create a study plan with specific deadlines and goals.**
* **Explore different study techniques to suit your learning style.**
* **Develop strategies to reduce test anxiety and improve your performance.**

I'm here to help you tackle this challenge, so don't hesitate to be specific!  The more information you give me, the better I can support you. 
